In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = "/content/drive/MyDrive/Projeto_Pneumonia/chest_xray/train"
test_dir = "/content/drive/MyDrive/Projeto_Pneumonia/chest_xray/test"
val_dir = "/content/drive/MyDrive/Projeto_Pneumonia/chest_xray/val"

Mounted at /content/drive


In [ ]:
#Gerador de imagens pré-processamento
image_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    samplewise_center=True,
    samplewise_std_normalization=True
)

In [ ]:
#Gerador dados treinamento e validação
train_generator = image_generator.flow_from_directory(
    train_dir,
    batch_size=8,
    shuffle=True,
    class_mode='binary',
    target_size=(180, 180)
)

validation_generator = image_generator.flow_from_directory(
    val_dir,
    batch_size=1,
    shuffle=False,
    class_mode='binary',
    target_size=(180, 180)
)


Found 5218 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
#Resnet
resnet_base_model = ResNet50(
    input_shape=(180, 180, 3),
    include_top=False,
    weights='imagenet'
)

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
#Camadas personalizadas
resnet_model = tf.keras.Sequential([
    resnet_base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.6),
    Dense(128, activation="relu"),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

In [ ]:
#Compilando Resnet
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]
resnet_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=METRICS)

In [ ]:
# Treinando modelo

# Calcular o peso das classes com base na contagem
num_normal = len(os.listdir(os.path.join(train_dir, 'NORMAL')))
num_pneumonia = len(os.listdir(os.path.join(train_dir, 'PNEUMONIA')))

weight_for_0 = num_pneumonia / (num_normal + num_pneumonia)
weight_for_1 = num_normal / (num_normal + num_pneumonia)

class_weight = {0: weight_for_0, 1: weight_for_1}

history = resnet_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    class_weight=class_weight,
    steps_per_epoch=100,
    validation_steps=25
)

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.3096 - accuracy: 0.5970 - precision: 0.8239 - recall: 0.5889

100/100 [==============================] - 545s 5s/step - loss: 0.3096 - accuracy: 0.5970 - precision: 0.8239 - recall: 0.5889 - val_loss: 881.4910 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
100/100 [==============================] - 473s 5s/step - loss: 0.3472 - accuracy: 0.5025 - precision: 0.7366 - recall: 0.5127
Epoch 3/10
100/100 [==============================] - 476s 5s/step - loss: 0.2964 - accuracy: 0.5612 - precision: 0.7837 - recall: 0.5554
Epoch 4/10
100/100 [==============================] - 476s 5s/step - loss: 0.2939 - accuracy: 0.5663 - precision: 0.7603 - recall: 0.5791
Epoch 5/10
100/100 [==============================] - 482s 5s/step - loss: 0.2762 - accuracy: 0.5550 - precision: 0.8053 - recall: 0.5204
Epoch 6/10
100/100 [==============================] - 482s 5s/step - loss: 0.2249 - accuracy: 0.6700 - precision: 0.9050 - recall: 0.6298
Epoch 7/10
100/100 [==============================] - 477s 5s/step - loss: 0.1926 - ac

In [ ]:
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
train_precision = history.history['precision']
train_recall = history.history['recall']

val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']
val_precision = history.history['val_precision']
val_recall = history.history['val_recall']

print("Métricas de Treinamento:")
print(f"Loss: {train_loss[-1]:.4f}")
print(f"Acurácia: {train_accuracy[-1] * 100:.2f}%")
print(f"Precisão: {train_precision[-1] * 100:.2f}%")
print(f"Recall: {train_recall[-1] * 100:.2f}%")

print("\nMétricas de Validação:")
print(f"Loss: {val_loss[-1]:.4f}")
print(f"Acurácia: {val_accuracy[-1] * 100:.2f}%")
print(f"Precisão: {val_precision[-1] * 100:.2f}%")
print(f"Recall: {val_recall[-1] * 100:.2f}%")

Métricas de Treinamento:
Loss: 0.1484
Acurácia: 83.13%
Precisão: 95.20%
Recall: 81.09%

Métricas de Validação:
Loss: 881.4910
Acurácia: 50.00%
Precisão: 0.00%
Recall: 0.00%
